## 1️⃣ Imports

In [ ]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("darkgrid")

## 2️⃣ Paths Configuration

In [2]:
MODELS_DIR = "../models"

WITHOUT_SEG_DIR = os.path.join(MODELS_DIR, "without_segmentation")
WITH_SEG_DIR = os.path.join(MODELS_DIR, "with_segmentation")


## 3️⃣ Utility: Load Metrics Files

In [3]:
def load_metrics(folder):
    records = []
    for file in os.listdir(folder):
        if file.endswith("_metrics_.json") or file.endswith("_metrics.json"):
            with open(os.path.join(folder, file), "r") as f:
                data = json.load(f)
                data["model_file"] = file
                records.append(data)
    return records


## 4️⃣ Load All Metrics

In [4]:
metrics_no_seg = load_metrics(WITHOUT_SEG_DIR)
metrics_seg = load_metrics(WITH_SEG_DIR)

print("Models without segmentation:", len(metrics_no_seg))
print("Models with segmentation:", len(metrics_seg))


Models without segmentation: 0
Models with segmentation: 0


## 5️⃣ Accuracy Comparison Table

In [5]:
def extract_accuracy(metrics_list, label):
    rows = []
    for m in metrics_list:
        rows.append({
            "type": label,
            "accuracy": m["accuracy"]
        })
    return pd.DataFrame(rows)

df_no_seg = extract_accuracy(metrics_no_seg, "No Segmentation")
df_seg = extract_accuracy(metrics_seg, "With Segmentation")

df_all = pd.concat([df_no_seg, df_seg], ignore_index=True)
df_all


""


## 6️⃣ Accuracy Comparison Plot

In [ ]:
plt.figure(figsize=(8,5))
sns.boxplot(data=df_all, x="type", y="accuracy")
plt.title("Accuracy Comparison")
plt.show()


## 7️⃣ Best Model Selection

In [ ]:
best_no_seg = max(metrics_no_seg, key=lambda x: x["accuracy"], default=None)
best_seg = max(metrics_seg, key=lambda x: x["accuracy"], default=None)

print("Best WITHOUT segmentation accuracy:", 
      best_no_seg["accuracy"] if best_no_seg else "N/A")

print("Best WITH segmentation accuracy:", 
      best_seg["accuracy"] if best_seg else "N/A")


## 8️⃣ Confusion Matrix Visualization

In [ ]:
def plot_confusion_matrix(cm, title):
    plt.figure(figsize=(6,5))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.title(title)
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.show()

if best_no_seg:
    plot_confusion_matrix(best_no_seg["confusion_matrix"], 
                          "Best Model – No Segmentation")

if best_seg:
    plot_confusion_matrix(best_seg["confusion_matrix"], 
                          "Best Model – With Segmentation")


## 9️⃣ Per-Class Metrics (Best Model)

In [ ]:
def classification_report_df(report_dict):
    df = pd.DataFrame(report_dict).transpose()
    return df.iloc[:-3]  # remove accuracy / avg rows

if best_seg:
    report_df = classification_report_df(best_seg["classification_report"])
    report_df
